In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
from sklearn.cluster import OPTICS, cluster_optics_dbscan, DBSCAN
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import multiprocessing
from joblib import Parallel, delayed
#import seaborn as sns
import pandas as pd

import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from matplotlib.collections import PatchCollection
import matplotlib.gridspec as gridspec


from astropy import units as u
from astropy.coordinates import SkyCoord

import warnings
warnings.filterwarnings('ignore')

import loaders

# MST groups

In [5]:
groups_mst, galxs_mst, branches = loaders.load_mst('Catalogos/MST/clusparam.dat','Catalogos/MST/miensalclumag.dat','Catalogos/MST/branches.txt')

AttributeError: module 'loaders' has no attribute 'load_mst'

In [ ]:
groups_mst.head(10)

## Catálogo

In [ ]:
df = pd.read_csv("Catalogos/final_cat_v4_October_myver.txt",delim_whitespace=True)

In [ ]:
c = SkyCoord(ra=df['ALPHA_J2000'], dec=df['DELTA_J2000'], frame='icrs',unit='degree')
df['l'] = c.galactic.l
df['b'] = c.galactic.b

## DUST

In [ ]:
dust = loaders.load_dust()

In [ ]:
def plot_branches(ax):
    for k,row in branches.iterrows():
        ax.plot([row['l1'],row['l2']],[row['b1'],row['b2']],'k')

In [ ]:
def plot_grupos(ax):
    unique_labels = groups_mst['nmem'].values
    norm = mcolors.LogNorm(4,91)
    colors = [plt.cm.jet(each) for each in norm(unique_labels)]
    
    for k, row in groups_mst.iterrows():
        radius = row['radius']
        circle = mpatches.Circle((row['l'],row['b']),lw=2,radius=radius/60,
                             edgecolor=plt.cm.jet(norm(row['nmem'])),facecolor='None')
        ax.add_patch(circle)
    return(norm)

In [ ]:
fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(211,aspect='equal')

plot_grupos(ax)
plot_branches(ax)
ax.contour(dust['xi'], dust['yi'], dust['zi'],levels=[11,15,20,25],extent=(df['l'].min(),df['l'].max(),df['b'].min(),df['b'].max()),origin='lower',alpha=0.6,cmap='gist_yarg')


ax.set_xlim(df['l'].max(),322.0)
ax.set_ylim(df['b'].min(),df['b'].max())
ax.set_xlabel('Galactic Longitude [deg]',fontsize=16)
ax.set_ylabel('Galactic Latitude [deg]',fontsize=16);
ax.tick_params(labelsize=16, width=3)
ax.grid(True)

ax = fig.add_subplot(212,aspect='equal')

ax.contour(dust['xi'], dust['yi'], dust['zi'],levels=[11,15,20,25],extent=(df['l'].min(),df['l'].max(),df['b'].min(),df['b'].max()),origin='lower',alpha=0.6,cmap='gist_yarg')

plot_branches(ax)
norm = plot_grupos(ax)

ax.set_xlim(322.0,df['l'].min())
ax.set_ylim(df['b'].min(),df['b'].max())
ax.set_xlabel('Galactic Longitude [deg]',fontsize=16)
ax.set_ylabel('Galactic Latitude [deg]',fontsize=16);
ax.tick_params(labelsize=16, width=3)
ax.grid(True)

#plt.subplots_adjust(bottom=0.1,right=0.8,top=0.9)
cax = plt.axes([0.93, 0.15, 0.01, 0.7])
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=plt.cm.jet),cax=cax,ticks=[4,10,30,91])
cbar.ax.set_yticklabels(['4','10','30','91'])
cbar.set_label('Number of members')